In [1]:
#on importe toutes les bibliothèques
import pandas as pd

In [2]:
train_data=pd.read_csv("data/train.csv")
X_test=pd.read_csv("data/test.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
#on créer une feature qui représente le groupe et une leur numéro du groupe
train_data["group"]=train_data["PassengerId"].str.slice(stop=4)
X_test["group"]=X_test["PassengerId"].str.slice(stop=4)

train_data["number"]=train_data["PassengerId"].str.slice(start=5)
X_test["number"]=X_test["PassengerId"].str.slice(start=5)

train_data['totalTips']=train_data['RoomService']+train_data['FoodCourt']+train_data['ShoppingMall']+train_data['Spa']+train_data['VRDeck']
X_test['totalTips']=X_test['RoomService']+X_test['FoodCourt']+X_test['ShoppingMall']+X_test['Spa']+X_test['VRDeck']

train_data["AgeBucket"] = train_data["Age"] // 15 * 15 #on créer un age bucket
train_data=train_data.merge(train_data.groupby(by=['group']).count().rename(columns={"PassengerId": "numberInGroup"})["numberInGroup"],on="group")

X_test["AgeBucket"] = X_test["Age"] // 15 * 15 #on créer un age bucket
X_test=X_test.merge(X_test.groupby(by=['group']).count().rename(columns={"PassengerId": "numberInGroup"})["numberInGroup"],on="group")

train_data=train_data.merge(train_data.groupby(by=['Cabin']).count().rename(columns={"PassengerId": "numberInCabin"})["numberInCabin"],on="Cabin")
X_test=X_test.merge(X_test.groupby(by=['Cabin']).count().rename(columns={"PassengerId": "numberInCabin"})["numberInCabin"],on="Cabin")

In [5]:
print(train_data["number"].nunique())
print(train_data["group"].nunique())

8
6118


on remarque qu'il y a beaucoup trop de groupe pour utiliser le onehot encoder, mais on peut utiliser les numbers

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [7]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
#pour l'instant on n'utilise pas tout, on se contente pour l'instant du plus simple
num_attribs = ["Age","RoomService", "FoodCourt", "ShoppingMall","Spa","VRDeck","totalTips","numberInGroup","numberInCabin"]
cat_attribs = ["HomePlanet", "CryoSleep", "Destination","VIP"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

clf = Pipeline(
    steps=[("preprocessor", preprocess_pipeline), ("classifier", SVC())]
)

X_train=train_data.copy()
X_train=X_train.drop(columns=["Transported"])
y_train=train_data["Transported"]

In [9]:
from sklearn.model_selection import cross_val_score
dt_scores = cross_val_score(clf, X_train, y_train, cv=5)
dt_scores.mean()

0.7826715777520346

In [12]:
clf.fit(X_train,y_train)

ValueError: A given column is not a column of the dataframe

In [32]:
y_pred=clf.predict(X_test)

In [ ]:
y_pred_df=pd.DataFrame(y_pred, columns = ['Transported'])
y_pred_df=y_pred_df.set_index(X_test["PassengerId"])

In [ ]:
y_pred_df.to_csv("C:/Users/miaou/Documents/UTT/ISI-2/PE_IA_RM/Python_script/spaceship_titanic_competition/data/y_pred.csv")

après avoir fait le test sur les modèles suivants : Random forest classique(0.7873010757740087),Gaussian nb(0.7361125761071896),Logistique regression(0.785460209860964),Plus proches voisins(0.7632582512733335),SVC(0.7889130493762866),Linear SVC(0.7845402732017329),Decision tree(0.7391004181470111), Bagging decison tree(0.7795942405691935)

les modèles les plus parformants sont alors : random forest, logistique regression, svc, linear svc

le modèle le plus performant est le SVC

pour le premier test on obtient un score de 0.78512 d'accuracy avec le random forest quand on le poste sur kaggle

On cherche maintenant à obtimiser la pipeline des données

On importe une fonction qui permet de calculer le taux de survie en fonction d'une feature

In [ ]:
# First create a function which will calculate survival rate given the column which we want to calculate it on 
def survival_rate(df,feature_name):
    # First get total rows(passengers) group-wise
    group_count=df[[feature_name,'Transported']].groupby(feature_name).count().rename(columns={'Transported':'Passengers'})
    group_survived= df[[feature_name,'Transported']].groupby(feature_name).sum().rename(columns={'Transported':'Survived'})
    final_df= group_count.merge(group_survived,how="left",left_index=True,right_index=True)
    final_df['Survival_rate']=final_df['Survived']/final_df['Passengers']
    final_df.drop(columns={'Passengers','Survived'},inplace=True)
    return final_df

In [ ]:
survival_rate(train_data,'numberInGroup')